<a href="https://colab.research.google.com/github/FredLongo/DataMiningProject/blob/main/code/GetTwitterSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This first part is about processing the Twitter data to a format we can work with.   


# Project Overview

## Evaluation of Twiter stock sentement on price stock prices.

In this project we evaluate twiter tweet posts to see if tweets can act as leading indecators or laging indicaterst to stock prices.  We do this over a one day, three day, five day pre and post of tweet.  




# Aquiring Tweets

We targeted twitter tweets to data mine as a source for evaluation of sentament.  Twitter was chosen because many of the paper we reviewed took this approch.  We wanted to emulate some of their process.  However Twitter has changed it's policies over the past year to not allow twitter pulls for free to research.  As a result We needed to use a thired party product to pull a few months worth of tweets.  This third party was Apify (www.apify.com)  as a tool it can pull twiter feeds.


## Limiting Data set
During this process we thought about the diffrent approches to evaluation of stock.  We could look at all tweets and get sentenment with respect to the market as a whole.  However we found that their are so many ways to iterprete the tweets with respect to the market as a whole was difficalt.  We chose to limit our stocks to just the top 7 of the S&P 500.  By doing so we felt this this would be easerir to selectivly identify tweets with respect to the stock.  The stocks we chose are "Apple","Alphabet","Amazon","Microsoft","Meta","Nvidia" and "Tesla".  Using their Stock lable of "AAPL","GOOG","AMZN","MSFT","META","NVDA","TSLA"

We also just looked a a few months of tweets.  So as to limit the data set to just a sample timeframe.  

Twitter has many users(channels) that produce tweets. We recognize not all channels are intresed in producing relevent, relyable and consistent statements with respect to market or stocks.  So we limit the channals that we monitored to just a few key player that we though might be relevent in the marketspace of stock evaluations.  The channals we chose to evaluate are "@USNewsMoney", "@TMFStockAdvisor", "@ftfinancenews", "@Stocktwits", "@MadMoneyOnCNBC" and "@SquawkCNBC".  We found these to be popular that should fit our needs.




Loading tweet data.

In [4]:
import json
import csv


In [14]:
import requests

#This pulls in the Tweets data
#Tweets URL of the file to be downloaded

Tweets_url = 'https://drive.google.com/file/d/1s429__b-2Gptf2oN2g-t--1aNPkMiyU-/view?usp=drive_link'

Tweets_filename = 'Tweets.json'

# Send a GET request to the URL
Tweets_response = requests.get(Tweets_url)

# Ensure the request was successful
if Tweets_response.status_code == 200:
    # Write the content of the response to a file
    with open(Tweets_filename, 'wb') as file:
        file.write(Tweets_response.content)
else:
    print(f"Failed to download the file. Status code: {Tweets_response.status_code}")





In [6]:

#Stock Price URL of the file to be downloaded
StockPrice_url = 'https://drive.google.com/file/d/1Hk3Ca66ai_vAO14EFAD1AfOEk2A60E-U/view?usp=drive_link'

# Send a GET request to the URL
StockPrice_response = requests.get(StockPrice_url)

# Ensure the request was successful
if StockPrice_response.status_code == 200:
    # Write the content of the response to a file
    with open('StockPrice.csv', 'wb') as file:
        file.write(StockPrice_response.content)
else:
    print(f"Failed to download the file. Status code: {StockPrice_response.status_code}")


In [11]:

#with open("twiterscraper.json", "r") as f:

#infile = "/content/dataset_easy-twitter-search-scraper_2023-11-21_01-10-14-456.json" # This has Stock
#infile = "/content/dataset_easy-twitter-search-scraper_2023-11-21_18-49-26-401.json"  # username filters
#infile = '/content/dataset_easy-twitter-search-scraper_2023-11-22_01-36-29-112.json' # no username filter

# Data pulled from https://console.apify.com/actors/2s3kSMq7tpuC3bI6M/runs/cMp5HramffKLAmVSt#output
infile = f"/content/{Tweets_filename}"

print(infile)
with open(infile, "r") as f:
  data = json.load(f)



/content/Tweets.json


JSONDecodeError: ignored

In [ ]:
'''
Set up Username Counts
'''

username =["@USNewsMoney", "@TMFStockAdvisor", "@ftfinancenews", "@Stocktwits", "@MadMoneyOnCNBC", "@SquawkCNBC"]

cols = 2
rows = len(username)
username_counts = [[0 for _ in range(cols)] for _ in range(rows)]

# Fill the first column with values from the list
for i in range(rows):
    username_counts[i][0] = username[i]

# Printing the 2D array
#for row in username_counts:    print(row)


In [ ]:
'''
Set up selectQ Counts
'''
searchQuery = ["stock","Apple","Microsoft","Alphabet","Amazon","Nvidia","Tesla","GOOG","AMZN","AAPL","META","MSFT","NVDA","TSLA"]

cols = 2
rows = len(searchQuery)
searchQuery_counts = [[0 for _ in range(cols)] for _ in range(rows)]

# Fill the first column with values from the list
for i in range(rows):
    searchQuery_counts[i][0] = searchQuery[i]




# Evaluation of Twitter Sentament

At this point I want to add the code to do the sentiment alalisis.



In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax


# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']



In [ ]:
def preprocesses_tweet(tweet):
  tweet_words = []

  # replace
  for word in tweet.split(' '):
    # username
    if word.startswith('@') and len(word) > 1 :
      word = '@user'

    # http link
    elif word.startswith('http'):
      word = "http"

    elif word == '':
      continue

    tweet_words.append(word)

  tweet_proc = " ".join(tweet_words)
  return tweet_proc


In [ ]:
def get_tweet_sentiment(tweet_proc):
  # sentiment analysis
  encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
  # output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
  output = model(**encoded_tweet)

  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  maxposition = 0
  maxscore = 0

  for i in range(len(scores)):
    if maxscore <= scores[i]:
      maxposition = i
      maxscore = scores[i]

  return labels[maxposition]  , scores[maxposition]




In [ ]:
out_data = []
loop_count = -1

# Need to create a tweet loop

for x in data:
  loop_count+=1
  try:
    tweet_proc = preprocesses_tweet(x['text'])
    sentiment_label, sentiment_score = get_tweet_sentiment(tweet_proc)
  except Exception as e:
    print(f"exception on {str(loop_count)}")
    print(f"Text:",x['text'])
    continue

  out_data.append(
        {"timestamp":x['timestamp']
        ,"username":x['user']['username']
        ,"searchQuery":x['searchQuery']
        ,"sentement":sentiment_label
        ,"score":str(sentiment_score)
        ,"text":tweet_proc})



exception on 66
exception on 94
exception on 222
exception on 299
exception on 412
exception on 557
exception on 625
exception on 743
exception on 856
exception on 907
exception on 948


In [ ]:
csv_file_name = infile[:-5] + "_out.csv"

# Open the CSV file in write mode
with open(csv_file_name, mode='w', newline='') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)

    # Write the header row based on the keys in the JSON data
    header = out_data[0].keys()
    csv_writer.writerow(header)

    # Write each row of data from the JSON object
    for row in out_data:
        csv_writer.writerow(row.values())

# Aquiring Stock Prices

# Joining Stock Prices with respect to Tweets

# Analytics

# Conclution